In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))
data = pd.read_csv("../input/train.csv")
data.shape

# Any results you write to the current directory are saved as output.

In [ ]:
data.head()

In [ ]:
data[data.target == 1].head()

In [ ]:
data['target'].value_counts()/data.shape[0]*100

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
insincere_rows = data[data.target ==1]
wc = WordCloud(background_color = 'white').generate(' '.join(insincere_rows['question_text']))
plt.imshow(wc)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
insincere_rows = data[data.target == 0]
wc = WordCloud(background_color = 'white').generate(' '.join(insincere_rows['question_text']))
plt.imshow(wc)

In [ ]:
from sklearn.model_selection import train_test_split
train, validate = train_test_split(data, test_size = .3, random_state = 1)
train.shape, validate.shape

In [ ]:
type(train['question_text'])

In [ ]:
import nltk
def clean_sentance(doc, stopwords,stemmer):
    words = doc.split(' ')
    words_clean = [stemmer.stem(word) for word in words if word not in stopwords]
    return' '.join(words_clean)

def clean_documents(docs_raw):
    stopwords = nltk.corpus.stopwords.words('english')
    stemmer = nltk.stem.PorterStemmer()
    docs = docs_raw.str.lower().str.replace('[^a-z ]','')
    docs_clean = docs.apply(lambda doc: clean_sentance(doc,stopwords,stemmer))
    return docs_clean
train_docs_clean = clean_documents(train['question_text'])
train_docs_clean.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vactorizer = CountVectorizer(min_df = 10).fit(train_docs_clean)
dtm = vactorizer.transform(train_docs_clean)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_df = DecisionTreeClassifier(max_depth = 10).fit(dtm, train['target'])

In [ ]:
validate_docs_clean = clean_documents(validate['question_text'])
dtm_validate = vactorizer.transform(validate_docs_clean)
dtm_validate

In [ ]:
validate_pred = model_df.predict(dtm_validate)
from sklearn.metrics import f1_score
f1_score(validate['target'],validate_pred)

In [39]:
from sklearn.naive_bayes import MultinomialNB
model_nb = MultinomialNB().fit(dtm,train['target'])
validate_pred = model_nb.predict(dtm_validate)
f1_score(validate['target'],validate_pred)

0.5425968470252713

In [41]:
test =pd.read_csv('../input/test.csv')
test.head()

,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?


In [42]:
docs_clean = clean_documents(test.question_text)
dtm_test = vactorizer.transform(docs_clean)
dtm_test

<375806x19550 sparse matrix of type '<class 'numpy.int64'>'
	with 2220720 stored elements in Compressed Sparse Row format>

In [43]:
test_pred = model_nb.predict(dtm_test)

In [44]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
submission = pd.DataFrame({'qid': test['qid'],
                          'prediction': test_pred})
submission[['qid','prediction']].to_csv('submission.csv',index = False)